In [3]:
from faker import Faker
import names
import google.auth
from google.cloud.sql.connector import Connector
from google.auth.transport.requests import Request
import sqlalchemy
import random
import requests
import time
import csv
import pandas as pd
from bs4 import BeautifulSoup

In [1]:
def get_majors():
    url = "https://myillini.illinois.edu/Programs"
    data = requests.get(url)
    html = BeautifulSoup(data.text,'html.parser')
    results = html.find_all("p",class_="mb-0")
    majors = []
    for i in range(len(results)):
        str = results[i].text.replace("\n", "")
        majors.append(str)
    return majors

def get_classes():
    df = pd.read_csv("https://waf.cs.illinois.edu/discovery/gpa.csv")
    df.groupby("Course Title").agg("count").reset_index()
    classes = df["Course Title"][df["Year"]>=2021]
    courses = []
    for i in range(len(classes)):
        if classes[i] not in courses:
            courses.append(classes[i])
    return courses

In [4]:
MAJORS = get_majors()
CLASSES = get_classes()  
RESIDENCE = ["ISR", "Snyder","Wassaja","Scott","Hopkins","Nugent","Weston","Bousfield", "Suites at Third", "Skyline Tower","The Dean","The HERE","309 Green","FAR","PAR","Tower at Third"]
RSOS = ["Association of Computing Machinery", "Asian American Association", "Illini Esports", "iRobotics", "Cricket Club of Illinois", "Engineering Council", "Illini Biliards Club", "Illini Kendo Club", "Illini Powerlifting Club", "K-Project", "Men's Club Soccer"]
INTERESTS = ["Powerlifting", "Soccer", "Basketball", "Football","Badminton","Tennis","Table Tennis","Running","Cycling"]
print (MAJORS)

['Accountancy', 'Accountancy + Data Science', 'ACES Undeclared', 'Acting', 'Actuarial Science', 'Advertising', 'Aerospace Engineering', 'African American Studies', 'Agri-Accounting', 'Agribusiness, Markets & Management', 'Agricultural and Biological Engineering (ACES)', 'Agricultural and Biological Engineering (ENG)', 'Agricultural and Consumer Economics', 'Agricultural Communications', 'Agricultural Education', 'Agricultural Leadership, Education, and Communications', 'Agronomy', 'Animal Sciences', 'Anthropology', 'Architectural Studies', 'Art and Art History ', 'Art Education (K-12)', 'Art Undeclared ', 'Arts and Entertainment Technology ', 'Asian American Studies', 'Astronomy', 'Astronomy + Data Science', 'Astrophysics', 'Atmospheric Sciences', 'Biochemistry', 'Bioengineering', 'Biology', 'Brain and Cognitive Science', 'Business Undeclared', 'Chemical Engineering', 'Chemistry', 'Civil Engineering', 'Classics', 'Communication', 'Community Health', 'Comparative and World Literature', 

In [52]:
# ONLY RUN ALL OF THIS IF YOU ABSOLUTELY NEED TO; DATABASE IS ALREADY POPULATED WITH PREEXISTING VALUES
def generateTuple ():
    firstName, lastName = names.get_first_name(), names.get_last_name()
    netId = firstName.lower()[0] + lastName.lower() + str(random.randint(0, 9))
    residence = random.choice(RESIDENCE)
    majorId = random.randint(0, len(MAJORS) - 1)
    major = MAJORS[majorId]
        
    courses = ""
    # Generate 3 to 6 random distinct classes
    class_ixs = random.sample(range(0, len(CLASSES)), random.randint(3, 6))
    for ix in range (0, len(class_ixs)):
        if (ix != len(class_ixs) - 1):
            courses += CLASSES[class_ixs[ix]] + ", "
        else:
            courses += CLASSES[class_ixs[ix]]

    hobbies = ""
    # Generate up to 5 random distinct interests
    hobbyIds = random.sample(range(0, len(INTERESTS)), random.randint(0, 5)) 
    for ix in range (0, len(hobbyIds)):
        if (ix != len(hobbyIds) - 1):
            hobbies += INTERESTS[ix] + ", "
        else:
            hobbies += INTERESTS[ix]
    
    # Generate up to three RSOs
    rso_ids = random.sample(range(0, len(RSOS) - 1), random.randint(0, 3))

    # Password is just the reverse of the NetID
    return {"netId" : netId, "firstName" : firstName, "lastName" : lastName, "password" : netId[::-1], "majorId" : majorId, "major" : major, "residence" : residence,  "courses" : courses, "hobbyIds" : hobbyIds, "hobbies" : hobbies, "RSO_IDS" : rso_ids}
    
def generateTables():
    students = []
    major = []
    login = []
    interests = []
    RSOMembers = []
    seen = {}
    # Generate 7500 rows of data
    for ix in range (15000):
        row = generateTuple()
        if (row["netId"] not in seen):
            seen[row["netId"]] = True
            gradYear = random.randint(2021,2026)
            friends = random.sample(students,random.randint(1,6))
            student_tuple = (row["netId"], row["firstName"], row["lastName"], row["major"], row["residence"], gradYear, friends)
            major_tuple = (row["majorId"], row["netId"], row["courses"])
            login_tuple = (row["netId"], row["firstName"], row["lastName"], row["password"])
            RSO_ids = row["RSO_IDS"]
            for jx in range (0, len(RSO_ids)):
                RSOMembers.append((RSO_ids[jx], row["netId"], row["firstName"], row["lastName"]))
            
            interest_ids = row["hobbyIds"]
            for kx in range (0, len(interest_ids)):
                interests.append((interest_ids[kx], row["netId"], INTERESTS[interest_ids[kx]]))

            students.append(student_tuple)
            major.append(major_tuple)
            login.append(login_tuple)

    return students, major, login, interests, RSOMembers

def generateCSVs():
    data = generateTables()
    files = ["Students.csv", "Major.csv", "Login.csv", "Interests.csv", "RSOMembers.csv"]
    for ix in range (0, len(files)):
        with open ("./data/" + files[ix], 'w', newline='') as out:
            csv_out = csv.writer(out)
            for row in data[ix]:
                csv_out.writerow(row)

In [53]:
generateCSVs()